# Run the auto_qsar pipeline

The following notebook provides a simple guide on how to run the auto_qsar pipeline and generate predictions for external test sets. Users can also input the SMILES of a compound and get the predicted chloride transport activity. 

A more detailed guide to running the pipeline can also be found in our published book chapter: Ciallella, H. L.; Chung, E.; Russo, D. P.; Zhu, H. Automatic Quantitative Structure–Activity Relationship Modeling to Fill Data Gaps in High-Throughput Screening. In High-Throughput Screening Assays in Toxicology; Zhu, H., Xia, M., Eds.; Methods in Molecular Biology; Humana: New York, NY, 2022; Vol. 2474, pp 169–187. https://doi.org/10.1007/978-1-0716-2213-1_16. The guide below is taken directly from the book chapter.

This script accepts eight flagged arguments, in no required order:

1. The training set filename without .sdf ending (-ds). This input value must correspond to a file present in the project directory.
2. The features with which to train the QSAR models (-f). The valid inputs for calculating binary chemical fingerprints are as follows: ECFP6 to calculate ECFPs, FCFP6 to calculate FCFPs, and MACCS to calculate MACCS keys. Use rdkit to calculate continuous RDKit chemical descriptors.
3. The number of splits to use for cross-validation (-ns). This input value must be an integer between 1 and the number of chemicals in the training set file. However, commonly used values are 5 and 10.
4. The project directory’s environment variable (-ev). The environment variable here is AUTO_QSAR_CL_TRANSPORT.
5. The name of the sdf property containing the chemicals’ unique identifier (-nc).
6. The name of the sdf property containing the endpoint for modeling (-ep).
7. The fraction of the training set to be reserved as a test set (-ts). Use a value of 0 for this flagged argument to train, optimize, and cross-validate QSAR models with the complete training set instead of reserving a test set.

Example for building the QSAR models: 
- ```python build_models.py -ds ChlorideTransporters-TrainingSet-1 -f rdkit -ns 5 -ev AUTO_QSAR_CL_TRANSPORT -nc NSC# -ep Activity```

Example for getting predictions from the QSAR models:
- ```python make_predictions.py -ds ChlorideTransporters-TrainingSet-1 -f rdkit -ev AUTO_QSAR_CL_TRANSPORT -nc NSC# -ep Activity -ps drugbankdb -a rf```

<span style='color: black; background-color: yellow;'><b>Note</b>:
To reproduce the results from the paper, users can run all the cells below by going to the menu above and selecting Run --> Run All Cells. This process may take an hour or longer. If users only want to predict the activity of one compound, then run the cell below, cell 1 (Build models), and cell 5 by pressing Shift + Enter for each of these three cells. Users can then input the SMILES of the desired compound into the box to get the predicted chloride transport activity.</span>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
from joblib import load
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors
from molecules_and_features import calc_rdkit, calc_maccs
from ipywidgets import interact, widgets
from IPython.display import display, HTML
from consensus_metrics import calc_metrics

## 1. Build models using the auto_qsar pipeline

In [ ]:
# List of the 19 training sets
files = [f'ChlorideTransporters-TrainingSet-{x}' for x in range(1, 20)]
features = ['MACCS', 'rdkit']
name_col = 'NSC#'

# Build models for the 19 training sets with the MACCS and rdkit descriptors
for file in files:
    for f in features:
        ! python build_models.py -ds {file} -f {f} -ns 5 -ev AUTO_QSAR_CL_TRANSPORT -nc {name_col} -ep Activity

## 2. Compile the five-fold cross-validation performance results for all the models into one DataFrame and save to a CSV file

In [ ]:
# Configuration of variables
filename = 'ChlorideTransporters-TrainingSet'
algorithms = ['dnn', 'rf', 'svm', 'xgb']
endpoint = 'Activity'
name_col = 'NSC#'
feature = 'rdkit'
num_sets = 19

# Initialize results containers
stats = pd.DataFrame()
ccr = pd.DataFrame(index=[f'Set{x}' for x in range(1, num_sets + 1)],
                   columns=[alg.upper()  for alg in algorithms] + ['Consensus'])

# Loop through all the training sets
for i in range(1, num_sets + 1):
    preds = pd.DataFrame()
    set_name = f'Set{i}'

    for alg in algorithms:
        alg_upper = alg.upper()

        # Read model metrics and predictions
        result_file = f"results/{alg}_{filename}-{i}_{feature}_{endpoint}_None_5fcv_results.csv"
        pred_file = f"predictions/{alg}_{filename}-{i}_{feature}_{endpoint}_None_5fcv_predictions.csv"
        result_data = pd.read_csv(result_file, header=None, index_col=0)
        pred_data = pd.read_csv(pred_file, index_col=0)['Probability']

        # Store CCR and metrics
        ccr.at[set_name, alg_upper] = result_data.at['CCR', 1]
        stats[f'{set_name} - {alg_upper}'] = result_data[1]
        preds[f'Probability - {alg_upper}'] = pred_data

    # Calculate and store consensus predictions
    preds['Consensus'] = preds.mean(axis=1)
    consensus_metrics = calc_metrics(preds['Consensus'], f'{filename}-{i}.sdf', endpoint=endpoint, name_col=name_col)

    stats[f'{set_name} - Consensus'] = consensus_metrics[0]
    ccr.at[set_name, 'Consensus'] = consensus_metrics.at['CCR', 0]

# Format final metrics table
total_stats = stats.T
total_stats['Algorithm'] = list(total_stats.index.str.extract(r' - (\w+)$')[0])
total_stats['Set'] = list(total_stats.index.str.extract(r'^(Set\d+)')[0])

# Save outputs to CSV files
total_stats.to_csv(f'ChlorideTransporters-MultipleTrainingSets-5fcv_metrics_{feature}.csv')
ccr.to_csv(f'ChlorideTransporters-MultipleTrainingSets-5fcv_CCR_{feature}.csv')

In [ ]:
# This dataframe can be found as a CSV file in the auto_qsar_cl_transport folder as shown in the code above
total_stats

## 3. Validate models using the validation set and predict chloride transporters from DrugBank

In [ ]:
# Configuration of variables
training_sets = [f'ChlorideTransporters-TrainingSet-{x}' for x in range(1, 20)]
test = 'ExternalTest-noactivity'
# test = 'drugbankdb'
algorithms = ['dnn', 'rf', 'svm', 'xgb']
name_col = 'NSC#'

# Predict chloride transporters for the test sets
for train in training_sets:
    for alg in algorithms:
        ! python make_predictions.py -ds {train} -f rdkit -ev AUTO_QSAR_CL_TRANSPORT -nc {name_col} -ep Activity -ps {test} -a {alg}

<span style='color: black; background-color: yellow;'><b>Note</b>: Running the cell above will provide predictions for the external validation set mentioned in the paper. If users want to get individual model predictions for the DrugBank dataset, replace ```test = 'ExternalTest-noactivity'``` with ```test = 'drugbankdb'``` in the above cell. Generating predictions for the DrugBank dataset when running the cell may take up to a few hours.</span>

## 4. Compile external test set model performance metrics into one DataFrame and save to a CSV file

In [ ]:
# Configuration of variables
testname = 'ExternalTest-noactivity'
expname = 'ExternalTest'
filename = 'ChlorideTransporters-TrainingSet'
feature = 'rdkit'
endpoint = 'Activity'
name_col = 'NSC#'
algorithms = ['dnn', 'rf', 'svm', 'xgb']

# Load experimental data
exp = pd.read_csv(f'{expname}.csv', index_col=name_col)

# Initialize containers
all_metrics = []
all_preds = pd.DataFrame()

# Iterate over the training sets
for i in range(1, 20):
    set_name = f'Set{i}'
    preds = pd.DataFrame()

    for alg in algorithms:
        alg_upper = alg.upper()
        pred_path = f'predictions/{alg}_{testname}_{filename}-{i}_{feature}_{endpoint}_None_no_gaps.csv'
        pred_series = pd.read_csv(pred_path, index_col=0)['Activities']
        col_name = f'Probability - {set_name} - {alg_upper}'
        
        # Store individual predictions
        preds[col_name] = pred_series
        all_preds[col_name] = pred_series

        # Compute and store metrics
        metrics = calc_metrics(pred_series, f'{expname}.sdf', endpoint=endpoint, name_col=name_col)
        metrics.at['Set', 0] = set_name
        metrics.at['Model', 0] = alg_upper
        metrics.columns = [f'{set_name} - {alg_upper}']
        all_metrics.append(metrics)

    # Consensus for this set
    preds['Consensus'] = preds.mean(axis=1)
    consensus_metrics = calc_metrics(preds['Consensus'], f'{expname}.sdf', endpoint=endpoint, name_col=name_col)
    consensus_metrics.at['Set', 0] = set_name
    consensus_metrics.at['Model', 0] = 'Consensus'
    consensus_metrics.columns = [f'{set_name} - Consensus']
    all_metrics.append(consensus_metrics)

# Overall Consensus
all_preds['Overall Consensus'] = all_preds.mean(axis=1)
overall_metrics = calc_metrics(all_preds['Overall Consensus'], f'{expname}.sdf', endpoint=endpoint, name_col=name_col)
overall_metrics.at['Set', 0] = 'Overall'
overall_metrics.at['Model', 0] = 'Consensus'
overall_metrics.columns = ['Overall Consensus']
all_metrics.append(overall_metrics)

# Combine all metrics
ext_metrics = pd.concat(all_metrics, axis=1).T

# Save results to a CSV file
print(f"Compiled metrics for {len(ext_metrics)} models.")
ext_metrics.reset_index(drop=True, inplace=True)
ext_metrics.to_csv(f'ExternalTest-AllMetrics-{feature}.csv', index=False)

In [ ]:
ext_metrics

## 5. Run the cell below then input a SMILES string into the box to get the predicted chloride transport activity

In [ ]:
model_dir = 'models/'
algorithms = ['dnn', 'rf', 'svm', 'xgb']
feature = 'rdkit'

# Load the 76 models
model_files = [f for f in os.listdir(model_dir) if (f.endswith('.pkl') and feature in f)]
model_files = [file for file in model_files if any(alg in file for alg in algorithms)]

# Load all models into a list
models = []
for file in model_files:
    with open(os.path.join(model_dir, file), 'rb') as f:
        models.append(load(f))

print(f"Loaded {len(models)} models.")

# ---------------------------
# Feature generation function
# (adjust the variable feature above to either MACCS or rdkit)
# ---------------------------

def smiles_to_features(smiles, feature):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    if feature == 'rdkit':
        calculator = MoleculeDescriptors.MolecularDescriptorCalculator([desc[0] for desc in Descriptors.descList])
        descriptors = calculator.CalcDescriptors(mol)
    else:
        descriptors = [float(x) for x in MACCSkeys.GenMACCSKeys(mol)]
    return np.array(descriptors).reshape(1, -1)

# ---------------------------
# Prediction function
# ---------------------------

def predict_consensus(smiles):
    features = smiles_to_features(smiles, feature)
    if features is None:
        return "Invalid SMILES"

    preds = []
    for model in models:
        try:
            preds.append(model.predict_proba(features)[0][1])  # probability of being active
        except Exception as e:
            return f"Error with model prediction: {e}"

    consensus_score = np.mean(preds)
    label = "Active" if consensus_score >= 0.5 else "Inactive"
    return f"Predicted activity: <b>{label}</b> (Consensus score: {consensus_score:.3f})"

# ---------------------------
# Interactive interface
# ---------------------------

smiles_input = widgets.Text(
    value='CC(CC(C#N)(C1=CC=CC=C1)C2=CC=CC=C2)N3CCCCC3',
    placeholder='Enter SMILES string',
    description='SMILES:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)

output = widgets.Output()

def on_submit(change):
    with output:
        output.clear_output()
        result = predict_consensus(smiles_input.value)
        display(HTML(f"<p style='font-size:16px'>{result}</p>"))

smiles_input.observe(on_submit, names='value')

display(HTML("<h3>Chloride Transport Prediction from the RDKit Consensus Model</h3>"))
display(smiles_input, output)
